En este archivo aplicaremos un modelo de recomendacion basado en la similitud del coseno. Esta funcion esta asociada a un endpoint de nuestra api, pero debido a las limitaciones de memoria en el servidor, realizaremos este proceso localmente y crearemos un dataset especifico para realizar consultas en nuestro endpoint.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
df_games_complete=pd.read_parquet('processed_data/games.parquet')
df_games=df_games_complete[['id','genres','app_name','specs']]

In [ ]:
#Se rellenan valores nulos en la columna 'genres'
df_games['genres'].fillna(value='', inplace=True)
#Se crea una nueva columna 'genres_str' que contenga las listas de géneros como strings
df_games['genres_str'] = df_games['genres'].apply(lambda x: ' '.join(map(str, x)))

#Se rellenan valores nulos en la columna 'specs'
df_games['specs'].fillna(value='', inplace=True)
#Se crea una nueva columna 'specs_str' que contenga las listas de géneros como strings
df_games['specs_str'] = df_games['specs'].apply(lambda x: ' '.join(map(str, x)))

#Se crea una nueva columna combinando generos y specs
df_games['genres_specs_str'] = df_games['genres_str'] + ' ' + df_games['specs_str']

In [14]:
df_games.head()

,id,genres,app_name,specs,genres_str,specs_str,genres_specs_str
0,761140.0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,[Single-player],Action Casual Indie Simulation Strategy,Single-player,Action Casual Indie Simulation Strategy Single...
1,643980.0,"[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play Indie RPG Strategy,Single-player Multi-player Online Multi-Player...,Free to Play Indie RPG Strategy Single-player ...
2,670290.0,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Single-player, Multi-player, Online Multi-Pla...",Casual Free to Play Indie Simulation Sports,Single-player Multi-player Online Multi-Player...,Casual Free to Play Indie Simulation Sports Si...
3,767400.0,"[Action, Adventure, Casual]",弹炸人2222,[Single-player],Action Adventure Casual,Single-player,Action Adventure Casual Single-player
4,773570.0,"[Action, Casual, Indie, Sports]",Log Challenge,"[Single-player, Full controller support, HTC V...",Action Casual Indie Sports,Single-player Full controller support HTC Vive...,Action Casual Indie Sports Single-player Full ...


In [16]:
#Se eliminan las columnas que ya no vamos a utilizar
df_games.drop(columns=['genres','specs','genres_str','specs_str'],inplace=True)
df_games.head()

,id,app_name,genres_specs_str
0,761140.0,Lost Summoner Kitty,Action Casual Indie Simulation Strategy Single...
1,643980.0,Ironbound,Free to Play Indie RPG Strategy Single-player ...
2,670290.0,Real Pool 3D - Poolians,Casual Free to Play Indie Simulation Sports Si...
3,767400.0,弹炸人2222,Action Adventure Casual Single-player
4,773570.0,Log Challenge,Action Casual Indie Sports Single-player Full ...


In [12]:
#Se crea el objeto CountVectorizer
vectorizer = CountVectorizer(binary=True)

#Se crea la matriz de términos-documentos
genres_matrix = vectorizer.fit_transform(df_games['genres_specs_str']).toarray()


In [13]:
genres_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
#Se calcula la similitud del coseno entre los juegos
cosine_similarities = cosine_similarity(genres_matrix, genres_matrix)


In [18]:
#Se crea la funcion que devuelve las 5 recomendaciones de juego basandose en la similitud del coseno
def recommend(product_id):

    
    n=5
    #Se comprueba que ele id ingresado este en el dataframe
    if product_id not in df_games['id'].values:
        return "Juego no encontrado en la base de datos"

    #Se obtiene el índice del juego con el id proporcionado
    idx = df_games[df_games['id'] == product_id].index[0]

    #Se obtienen las puntuaciones de similitud del coseno para ese juego con respecto a todos los demás
    sim_scores = list(enumerate(cosine_similarities[idx]))

    #Se ordenan los juegos según sus puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    #Se obtienen los índices de los juegos recomendados
    sim_scores = sim_scores[1:n+1]  # Excluyendo el propio juego
    game_indices = [i[0] for i in sim_scores]

    #Devuelve la lista de juegos recomendados
    return df_games['app_name'].iloc[game_indices].tolist()



Dado que las limitaciones de memoria del sitio utilizado para hacer el deploy no me permiten realizar este proceso, se realiza en local y se crea una nueva columna con las recomendaciones.

In [19]:
#Se aplica la funcion a cada registro del dataframe y se guarda el resultado en una nueva columna
df_games['recommended_5'] = df_games['id'].apply(recommend)

In [20]:
df_games.head()

,id,app_name,genres_specs_str,recommended_5
0,761140.0,Lost Summoner Kitty,Action Casual Indie Simulation Strategy Single...,"[Wooden Battles, Bubonic: Outbreak, YOU ARE KI..."
1,643980.0,Ironbound,Free to Play Indie RPG Strategy Single-player ...,"[Duelyst, Animation Throwdown: The Quest for C..."
2,670290.0,Real Pool 3D - Poolians,Casual Free to Play Indie Simulation Sports Si...,"[Snooker-online multiplayer snooker game!, Her..."
3,767400.0,弹炸人2222,Action Adventure Casual Single-player,"[Biozone, Luxor: 5th Passage, CHASER, Turtle O..."
4,773570.0,Log Challenge,Action Casual Indie Sports Single-player Full ...,"[Wingless, Caretaker Retribution, Fall Fear Fl..."


Se guarda el dataframe con la nueva columna generada para utilizar en el endpoint de recomendacion

In [22]:
#Se elimina la columnas que ya no son necesarias y se guarda el df con las recomendaciones.
df_games.drop(columns=['genres_specs_str'],inplace=True)
df_games.to_parquet('API/games_recommendations.parquet')